The sampling is based on 0 < L < U, 0 < T uniformly

In [2]:
import random
from math import log
from math import floor
import numpy as np
from time import time
import sys
import json
from bisect import bisect_left

precision = 1000
timeInterval = 12*60*60 # half day

def getVLogic(logic):
    """Value logic"""
    vLogic = list(map(lambda x:int(x), logic.split('_')))
    return vLogic

def getLLogic(vLogic):
    """Log logic"""
    lLogic = list(map(lambda x: int(log(x,2)), vLogic))
    return lLogic

def rSample(vLogic, no):
    """The input is value logic"""
    lLogic = getLLogic(vLogic)
    samples = []
    for _ in range(no):
        temp = []
        for j in range(len(lLogic)):
            fsample =[sorted(random.sample(range(1,precision),2)) for _ in range(lLogic[j])]
            temp+=fsample
        samples.append(temp)
    return samples

def indexMask(vLogic):
    """Generate map i-> bit mask of i for value logic"""
    no = np.prod(vLogic)
    l = int(log(no,2))
    pattern = '{0:0'+str(l)+'b}'
    ret = []
    for i in range(no):
        mask = pattern.format(i)
        vMask = [int(i) for i in mask]
        currMask = []
        lLogic = getLLogic(vLogic)
        currPos = 0
        for i in range(len(lLogic)):
            temp = vMask[currPos:currPos+lLogic[i]]
            currMask.append(temp)
            currPos += lLogic[i]
        ret.append(currMask)
    return ret

def evalMask(sample, masks):
    """Given masks evaluate the sample sequence of switching polynomials"""
    ret = []
    for i in range(len(masks)):
        currVal = 1
        currPos = 0
        for j in range(len(masks[i])):
            tempSum = 0
            for k in range(len(masks[i][j])):
                if masks[i][j][k] == 0:
                    tempSum += sample[currPos][0]
                else:
                    tempSum += sample[currPos][1]
                currPos +=1
            currVal*=tempSum
        ret.append(currVal)
    return ret 


def reachTime(startTime,timeLimit):
    if time() - startTime > timeLimit:
        return True
    else:
        return False
    
def saveResult(logicName, fileNumber, res):
    fileName = logicName +'_'+str(fileNumber)
    with open(fileName,'w') as file:
        json.dump(res,file)
    
    return


            

def runSampleSPolynomials(logic):
    """Sample the swithcing polynomials without thetas"""
    roundLimit = 1000000
    
    res = dict()
    count = 1
    vLogic = getVLogic(logic)
    masks = indexMask(vLogic)
    start = time()

    while True:
        print(count)
        samples = rSample(vLogic,roundLimit)
        for i in range(len(samples)):
            tempVals = evalMask(samples[i],masks)
            if len(np.unique(tempVals)) != len(tempVals):
                continue
            tempOrd = str(np.argsort(tempVals))
            
            if tempOrd not in res:
                res[tempOrd] = 1
            else:
                res[tempOrd] +=1

        if reachTime(start, count*timeInterval):
            saveResult(str(vLogic), count, res)
            count+=1
            
def genThetas(noThetas,degree):
    """Generated Thetas, return thetas from the smallest to the biggest"""
    thetas = sorted(random.sample(range(1,precision**degree),noThetas))
    return thetas


def binSort(x):
    prev = 0
    for i in range(len(x)):
        if x[i] < 0:
            x[prev:i] = sorted(x[prev:i])
            prev = i+1
            
    l = len(x)
    x[prev:l] = sorted(x[prev:l])
    return x
            
    
            
def runSampleWithThetas(logic,noThetas):
    """Sample the swithcing polynomials without thetas"""
    roundLimit = 10000
    res = dict()
    count = 1
    vLogic = getVLogic(logic)
    masks = indexMask(vLogic)
    start = time()
    degree = len(vLogic)
    

    while True:
        print(count)
        samples = rSample(vLogic,roundLimit)
        for i in range(len(samples)):
            tempVals = evalMask(samples[i],masks)
            thetas = genThetas(noThetas,degree)
            if len(np.unique(tempVals+thetas)) != (len(tempVals)+len(thetas)):
                continue
            
            tempOrd = np.argsort(tempVals+thetas)
            for i in range(len(tempOrd)):
                if tempOrd[i] >= len(tempVals):
                    tempOrd[i] -= len(tempOrd)
                
            tempOrd = str(binSort(tempOrd))
            
            #print(tempord)
            if tempOrd not in res:
                res[tempOrd] = 1
            else:
                res[tempOrd] +=1

        if reachTime(start, count*timeInterval):
            saveResult(str(vLogic), count, res)
            count+=1

In [116]:
runSampleWithThetas('4_2',1)

1
2
3


In [104]:
runSample('4_2')

1
2
3


In [94]:
json.dump?

In [88]:
masks = indexMask([8])

In [89]:
sample = rSample([8],2)
sample

[[[459, 864], [241, 675], [127, 928]], [[242, 727], [18, 21], [634, 890]]]

In [90]:
evalMask(sample[0],masks)

[827, 1628, 1261, 2062, 1232, 2033, 1666, 2467]

In [83]:
masks

[[[0, 0, 0]],
 [[0, 0, 1]],
 [[0, 1, 0]],
 [[0, 1, 1]],
 [[1, 0, 0]],
 [[1, 0, 1]],
 [[1, 1, 0]],
 [[1, 1, 1]]]

In [ ]:
roundLimit = 1000000
timeInterval = 12*60*60
res = dict()
count = 1
vLogic = getVLogic(logic)
masks = indexMask(vLogic)
start = time()

while True:
    count+=1
    samples = rSample(vLogic,roundLimit)
    for i in range(len(samples)):
        tempVals = evalMask(samples[i],masks)
        if len(np.unique(tempvals)) != len(tempvals):
            continue
        tempOrd = str(np.argsort(tempVals))
        #print(tempord)
        if tempOrd not in res:
            res[tempOrd] = 1
        else:
            res[tempOrd] +=1
    
    if reachTime(start, count*timeInterval):
        saveResult(str(vLogic), count, res)
    

In [20]:
start = time()
logic = '4_2'
noThetas = 2

roundLimit = 10000

count = 1
vLogic = getVLogic(logic)
masks = indexMask(vLogic)
start = time()
degree = len(vLogic)
res=dict()

while count < 2:
    print(count)
    samples = rSample(vLogic,roundLimit)
    for i in range(len(samples)):
        tempVals = evalMask(samples[i],masks)
        
        if len(np.unique(tempVals)) != len(tempVals):
            continue

        thetas = genThetas(noThetas,degree)
        tempOrd = np.argsort(tempVals+thetas)
        for i in range(len(tempOrd)):
            if tempOrd[i] >= len(tempVals):
                tempOrd[i] -= len(tempOrd)

        tempOrd = str(binSort(tempOrd))

        #print(tempord)
        if tempOrd not in res:
            res[tempOrd] = 1
        else:
            res[tempOrd] +=1
        
    if reachTime(start, count*timeInterval):
        saveResult(str(vLogic), count, res)
        count+=1
    count+=1
    print(len(res))

time()-start

1
154


1.7412631511688232

In [8]:
samples = rSample(vLogic,roundLimit)

In [21]:
samples[0]

[[99, 677], [100, 813], [143, 519]]

In [23]:
tempVals

[8400, 86100, 40440, 414510, 33160, 339890, 65200, 668300]

In [22]:
thetas

[30013, 335418]

In [24]:
tempOrd = np.argsort(tempVals+thetas)
for i in range(len(tempOrd)):
    if tempOrd[i] >= len(tempVals):
        tempOrd[i] -= len(tempOrd)

tempOrd = str(binSort(tempOrd))

In [25]:
tempOrd

'[ 0 -2  1  2  4  6 -1  3  5  7]'

In [60]:
def generateRegionForNode(logic, samples, thetas):
    
    vLogic = getVLogic(logic)
    masks = indexMask(vLogic)
    
    tempVals = evalMask(samples,masks)
    
    if len(np.unique(tempVals)) != len(tempVals):
            return False
    
    rthetas = thetas.copy()[::-1]
    tempOrd = np.argsort(tempVals+rthetas)
    for i in range(len(tempOrd)):
        if tempOrd[i] >= len(tempVals):
            tempOrd[i] -= len(tempOrd)
    tempOrd = str(list(binSort(tempOrd)))
    return eval(tempOrd)

def generateRegionForNetwork(logics,sampleList,thetaList):
    ret = []
    for i in range(len(logics)):
        logic, samples, thetas = logics[i], sampleList[i], thetaList[i]
        factor = generateRegionForNode(logic, samples, thetas)
        ret.append(factor)
    return ret

In [66]:
# exmaple of toggle switch
# generate the inputs
logics=['2','2']
vLogic = getVLogic('2')

sampleList = rSample(vLogic,2)
thetaList = [genThetas(1,1) for _ in range(len(logics))]

In [70]:
generateRegionForNetwork(logics,sampleList,thetaList)

[[-1, 0, 1], [-1, 0, 1], [0, -1, 1]]

In [71]:
# example of repressilator 
logics=['2','2','2']
vLogic = getVLogic('2')

sampleList = rSample(vLogic,3)
thetaList = [genThetas(1,1) for _ in range(len(logics))]

In [72]:
generateRegionForNetwork(logics,sampleList,thetaList)

[[-1, 0, 1], [0, -1, 1], [-1, 0, 1]]

In [73]:
# example of p53
# example of repressilator 
logics=['2','2','2','2_2','4_2']
vLogics = [getVLogic(x) for x in logics]

sampleList = [rSample(vLogic,1)[0] for vLogic in vLogics]
thetaList = [genThetas(1,1), genThetas(2,1), genThetas(2,1), genThetas(1,1), genThetas(2,1) ]

In [74]:
generateRegionForNetwork(logics,sampleList,thetaList)

[[-1, 0, 1],
 [0, -2, 1, -1],
 [0, -2, -1, 1],
 [-1, 0, 1, 2, 3],
 [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7]]